In [ ]:
import numpy as np
import tensorflow as tf;
from tensorflow.contrib.tensorboard.plugins import projector;
from tensorflow.examples.tutorials.mnist import input_data;

# 载入数据集
mnist = input_data.read_data_sets("MNIST_data",one_hot = True)


# 运行次数
max_steps = 1001;

# 图片数量
image_num = 3000;

# 文件路径
DIR = "E:/TensorflowCode/Jupyter/TensorflowLearningAndApplication/"

# 定义会话
sess = tf.Session()

# 载入图片
# tf.stack 降维度操作，这里就是将多张图片和成一个矩阵
embedding = tf.Variable(tf.stack(mnist.test.images[:image_num]),trainable = False, name = "embedding")

# 参数概要
def variable_summaries(var):
    with tf.name_scope("summaries"):
        mean = tf.reduce_mean(var)
        tf.summary.scalar("mean",mean); #平均值
        with tf.name_scope("summaries"):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar("stddev",stddev) #标准差
        tf.summary.scalar("max",tf.reduce_max(var)) #最大值
        tf.summary.scalar("min",tf.reduce_min(var)) #最小值
        tf.summary.histogram("histogram",var)#直方图


# 定义命名空间
with tf.name_scope("input"):
    # 定义图片的站位数据
    x = tf.placeholder(tf.float32,[None,784],name="x-input")
    y = tf.placeholder(tf.float32,[None,10],name="y-input")
        

# 显示图片
with tf.name_scope("input_reshape"):
#         这里将一张图片一行的数据，转换成28*28的向量，同时第一个参数为-1表示数据不确定，最后一个参数为1表示数据是黑白图片
        image_shaped_input = tf.reshape(x,[-1,28,28,1])
        tf.summary.image("input",image_shaped_input,10)

with tf.name_scope("layer"):
    # 创建神经网络
    with tf.name_scope("wights"):
        W = tf.Variable(tf.ones([784,10]),name="W")
        variable_summaries(W)
    with tf.name_scope("biases"):
        b = tf.Variable(tf.ones([10]),name = "b")
        variable_summaries(b)
    with tf.name_scope("wx_plus_b"):
        wx_plus_b = tf.matmul(x,W)+b
    with tf.name_scope("softmax"): 
        perdiction = tf.nn.softmax(wx_plus_b)
        
with tf.name_scope("loss"): 
    # 交叉熵优化
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits = perdiction))
    tf.summary.scalar("loss",loss)
    
    
with tf.name_scope("train_setp"): 
    # 使用梯度下降
    train_setp = tf.train.GradientDescentOptimizer(0.22).minimize(loss)


init = tf.global_variables_initializer()

with tf.name_scope("accuracy"): 
    with tf.name_scope("correct_prediction"): 
        # 将返回最大的值放在一个一维张量中
        correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(perdiction,1))# argmax 返回一维张量中最大的值所在的位置
    with tf.name_scope("accuracy"): 
        # 测试准确率
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))# 把 correct_prediction 变为 float32 类型
        tf.summary.scalar("accuracy",accuracy)

        
# 生成metadata文件        
if tf.gfile.Exists(DIR+"projector/projector/metadata.tsv"):
    tf.gfile.DeleteRecursively(DIR + "projector/projector/metadata.tsv")
with open(DIR + "projector/projector/metadata.tsv" ,"w") as f:
    lables = sess.run(tf.arg_max(mnist.test.labels[:],1))
    for i in range(image_num):
        f.write(str(lables[i])+"\n")

        
# 合并所有检测的数据
merged = tf.summary.merge_all();

# 写入定义的网络图的结构
project_writer = tf.summary.FileWriter(DIR + "projector/projector" , sess.graph)
saver = tf.train.Saver()
config = projector.ProjectorConfig()
embed = config.embeddings.add()
embed.tensor_name = embedding.name;
embed.metadata_path = DIR + "projector/projector/metadata.tsv"
embed.sprite.image_path = DIR + "projector/data/mnist_10k_sprite.png"
# 设置图片拆分的方式
embed.sprite.single_image_dim.extend([28,28])
# 载入配置
projector.visualize_embeddings(project_writer,config)


for i in range(max_steps):
#     每次训练100个样本
    batch_xs,batch_ys = mnist.train.next_batch(100)
    run_options = tf.RunOptions(trace_level = tf.RunOptions.FULL_TRACE)
    run_metadata = tf.RunMetadata()
#     summary,_ = sess.run([merged ,train_setp],feed_dict= {x : batch_xs ,y: batch_ys},options= run_options ,run_metadata= run_metadata)
    summary,_  = sess.run([merged, train_step], feed_dict={x:batch_xs,y:batch_ys}, options=run_options, run_metadata=run_metadata)
    project_writer.add_run_metadata(run_metadata,"step%03d" % i)
    project_writer.add_summary(summary,i)
    
    if i%100 == 0:
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print("Iter" + str(epoch) + ",Testing Acc"+ str(acc))

saver.save(sess,DIR + "projector/projector/a_model.ckpt",global_step=max_steps)
project_writer.close()
sess.close()

    

W0827 16:41:04.302578  7220 deprecation.py:323] From <ipython-input-1-13ca4e30102e>:7: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0827 16:41:04.308570  7220 deprecation.py:323] From E:\TensorFlow\envs\tensorflow\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0827 16:41:04.310071  7220 deprecation.py:323] From E:\TensorFlow\envs\tensorflow\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updati

Extracting MNIST_data\train-images-idx3-ubyte.gz


W0827 16:41:04.563755  7220 deprecation.py:323] From E:\TensorFlow\envs\tensorflow\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0827 16:41:04.566257  7220 deprecation.py:323] From E:\TensorFlow\envs\tensorflow\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0827 16:41:04.616293  7220 deprecation.py:323] From E:\TensorFlow\envs\tensorflow\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed i

Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
